# BIDS Loader


We use the data in `SpineGeneric`

In [20]:
from bids_neuropoly import bids
from medicaltorch import datasets as mt_datasets


dct_label_map = { "laacq-MToff_MTS" : 0, "acq-MTon_MTS" : 1, 
                 "acq-T1w_MTS" : 2, "T1w" : 3, "T2star" : 4, "T2w" : 5}


class BIDSSegPair2D(mt_datasets.SegmentationPair2D):
    def __init__(self, input_filename, gt_filename, metadata):
        super().__init__(input_filename, gt_filename)
        self.metadata = metadata
        self.metadata["input_filename"] = input_filename
        self.metadata["gt_filename"] = gt_filename

    def get_pair_slice(self, slice_index, slice_axis=2):
        dreturn = super().get_pair_slice(slice_index, slice_axis)
        self.metadata["slice_index"] = slice_index
        dreturn["input_metadata"]["bids_metadata"] = self.metadata
        return dreturn


class MRI2DBidsSegDataset(mt_datasets.MRI2DSegmentationDataset):
    def _load_filenames(self):
        for input_filename, gt_filename, bids_metadata in self.filename_pairs:
            segpair = BIDSSegPair2D(input_filename, gt_filename,
                                    bids_metadata)
            self.handlers.append(segpair)


class BidsDataset(MRI2DBidsSegDataset):
    def __init__(self, root_dir, slice_axis=2, cache=True,
                 transform=None, slice_filter_fn=None,
                 canonical=False, labeled=True):
        self.bids_ds = bids.BIDS(root_dir)
        self.filename_pairs = []

        for subject in self.bids_ds.get_subjects():

            derivatives = subject.get_derivatives("labels")
            cord_label_filename = None

            for deriv in derivatives:
                if deriv.endswith("seg-manual.nii.gz"):
                    cord_label_filename = deriv

            if cord_label_filename is None:
                continue

            metadata = subject.metadata()
            self.filename_pairs.append((subject.record.absolute_path,
                                        cord_label_filename, metadata))
            
        super().__init__(self.filename_pairs, slice_axis, cache,
                         transform, slice_filter_fn, canonical)

# Model

Here we define the architecture of the network in a PyTorch Module

In [4]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data.sampler import SubsetRandomSampler



class DownConv(nn.Module):
    def __init__(self, in_ch, out_ch, bn_momentum=0.1):
        super(DownConv, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3, stride=1, padding=1)
        self.conv_bn = nn.BatchNorm2d(out_ch, momentum=bn_momentum)
    
    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.conv_bn(x)
        return x

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class Classifier(nn.Module):
    def __init__(self, drop_rate=0.4, bn_momentum=0.1):
        super(Classifier, self).__init__()
        self.conv1 = DownConv(1, 32, bn_momentum)
        self.mp1 = nn.MaxPool2d(2)
        
        self.conv2 = DownConv(32, 32, bn_momentum)
        self.mp2 = nn.MaxPool2d(2)
        
        self.conv3 = DownConv(32, 64, bn_momentum)
        self.mp3 = nn.MaxPool2d(2)       
        
        self.flat = Flatten()
        self.dense1 = nn.Linear(16384, 256)
        self.drop = nn.Dropout2d(drop_rate)
        self.dense2 = nn.Linear(256, 6)
        self.soft = nn.Softmax()

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.mp1(x1)
        
        x3 = self.conv2(x2)
        x4 = self.mp2(x3)
        
        x5 = self.conv3(x4)
        x6 = self.mp3(x5)
        
        x7 = self.flat(x6)
        x8 = self.dense1(x7)
        x9 = self.drop(x8)
        x10 = self.dense2(x9)
        x11 = self.soft(x10)
        
        return(x11)

#torch tensors are of the format (batch_size, n_channels, shape_of_image)
#a = torch.rand(100,1,128,128)
#print(Classifier().forward(a).shape)

# Training the model

In [18]:
from tensorboardX import SummaryWriter
import time
import shutil
import sys
import pickle
import nibabel as nib
import numpy as np
import json
import os
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from skimage.transform import resize

def get_modality(batch):
    labels = []
    for acq in batch['input_metadata']:
        path = acq.__getitem__('bids_metadata')['input_filename']
        name = os.path.basename(path)
        if "acq-MToff_MTS" in name :
            labels.append(0)
            continue
        if "acq-MTon_MTS" in name :
            labels.append(1)
            continue
        if "acq-T1w_MTS" in name :
            labels.append(2)
            continue
        if "T1w" in name :
            labels.append(3)
            continue
        if "T2star" in name :
            labels.append(4)
            continue
        if "T2w" in name :
            labels.append(5) 
            continue
    return labels

def OneHotEncode(labels):
    ohe_labels = []
    for label in labels :
        ohe = [0 for i in range(6)]
        ohe[label] = 1 
        ohe_labels.append(ohe)
    return torch.FloatTensor(ohe_labels)


def cmd_train(context):
    """Main command do train the network.
    :param context: this is a dictionary with all data from the
                    configuration file:
                        - 'command': run the specified command (e.g. train, test)
                        - 'gpu': ID of the used GPU
                        - 'bids_path_train': list of relative paths of the BIDS folders of each training center
                        - 'bids_path_validation': list of relative paths of the BIDS folders of each validation center
                        - 'bids_path_test': list of relative paths of the BIDS folders of each test center
                        - 'batch_size'
                        - 'dropout_rate'
                        - 'batch_norm_momentum'
                        - 'num_epochs'
                        - 'initial_lr': initial learning rate
                        - 'log_directory': folder name where log files are saved
                        - 'debugging': allows extended verbosity and intermediate outputs
    """
    # Set the GPU
    gpu_number = int(0)
    torch.cuda.set_device(gpu_number)

    # These are the training transformations
    train_transform = transforms.Compose([
        mt_transforms.CenterCrop2D((128, 128)),
        mt_transforms.ElasticTransform(alpha_range=(28.0, 30.0),
                                       sigma_range=(3.5, 4.0),
                                       p=0.3),
        mt_transforms.RandomAffine(degrees=4.6,
                                   scale=(0.98, 1.02),
                                   translate=(0.03, 0.03)),
        mt_transforms.RandomTensorChannelShift((-0.10, 0.10)),
        mt_transforms.ToTensor(),
        mt_transforms.NormalizeInstance(),
    ])

    # These are the validation/testing transformations
    val_transform = transforms.Compose([
        mt_transforms.CenterCrop2D((128, 128)),
        mt_transforms.ToTensor(),
        mt_transforms.NormalizeInstance(),
    ])

    # This code will iterate over the folders and load the data, filtering
    # the slices without labels and then concatenating all the datasets together
    train_datasets = []
    for bids_ds in tqdm_notebook(context["bids_path_train"], desc="Loading training set"):
        ds_train = BidsDataset(bids_ds,
                               transform=train_transform)
        train_datasets.append(ds_train)

    ds_train = ConcatDataset(train_datasets)
    print(f"Loaded {len(ds_train)} axial slices for the training set.")
    train_loader = DataLoader(ds_train, batch_size=context["batch_size"],
                              shuffle=True, pin_memory=True,
                              collate_fn=mt_datasets.mt_collate,
                              num_workers=1)

    # Validation dataset ------------------------------------------------------
    validation_datasets = []
    for bids_ds in tqdm_notebook(context["bids_path_validation"], desc="Loading validation set"):
        ds_val = BidsDataset(bids_ds,
                             transform=val_transform)
        validation_datasets.append(ds_val)

    ds_val = ConcatDataset(validation_datasets)
    print(f"Loaded {len(ds_val)} axial slices for the validation set.")
    val_loader = DataLoader(ds_val, batch_size=context["batch_size"],
                            shuffle=True, pin_memory=True,
                            collate_fn=mt_datasets.mt_collate,
                            num_workers=1)

    model = Classifier(drop_rate=context["dropout_rate"],
                       bn_momentum=context["batch_norm_momentum"])
    model.cuda()

    num_epochs = context["num_epochs"]
    initial_lr = context["initial_lr"]

    # Using Adam with cosine annealing learning rate
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)

    # Write the metrics, images, etc to TensorBoard format
    writer = SummaryWriter(log_dir=context["log_directory"])

    # Training loop -----------------------------------------------------------
    best_validation_loss = float("inf")
    for epoch in tqdm_notebook(range(1, num_epochs+1), desc="Training"):
        start_time = time.time()

        scheduler.step()

        lr = scheduler.get_lr()[0]
        writer.add_scalar('learning_rate', lr, epoch)

        model.train()
        train_loss_total = 0.0
        num_steps = 0
        
        for i, batch in enumerate(train_loader):
            input_samples = batch["input"]
            input_labels = get_modality(batch)
            
            var_input = input_samples.cuda()
            var_labels = OneHotEncode(input_labels).cuda()

            preds = model(var_input)

            CE_loss = nn.BCELoss()
            loss = CE_loss(preds, var_labels)
            train_loss_total += loss.item()

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            num_steps += 1

        train_loss_total_avg = train_loss_total / num_steps

        #tqdm.write(f"Epoch {epoch} training loss: {train_loss_total_avg:.4f}.")
        print(f"Epoch {epoch} training loss: {train_loss_total_avg:.4f}.")
        
        # Validation loop -----------------------------------------------------
        model.eval()
        val_loss_total = 0.0
        num_steps = 0

        for i, batch in enumerate(val_loader):
            input_samples = batch["input"]
            input_labels = get_modality(batch)
            
            with torch.no_grad():
                var_input = input_samples.cuda()
                var_labels = OneHotEncode(input_labels).cuda()

                preds = model(var_input)

                CE_loss = nn.BCELoss()
                loss = CE_loss(preds, var_labels)
                val_loss_total += loss.item()

            num_steps += 1

        val_loss_total_avg = val_loss_total / num_steps

        #tqdm.write(f"Epoch {epoch} validation loss: {val_loss_total_avg:.4f}.")
        print(f"Epoch {epoch} validation loss: {val_loss_total_avg:.4f}.")
        
        end_time = time.time()
        total_time = end_time - start_time
        #tqdm.write("Epoch {} took {:.2f} seconds.".format(epoch, total_time))
        print("Epoch {} took {:.2f} seconds.".format(epoch, total_time))
        
        if val_loss_total_avg < best_validation_loss:
            best_validation_loss = val_loss_total_avg
            torch.save(model, "./"+context["log_directory"]+"/best_model.pt")

    # save final model
    torch.save(model, "./"+context["log_directory"]+"/final_model.pt")
    
    return


def run_main(command):
    with open('config_small.json') as fhandle:
        context = json.load(fhandle)

    #command = context["command"]

    if command == 'train':
        cmd_train(context)
        shutil.copyfile(sys.argv[1], "./"+context["log_directory"]+"/config_file.json")
    elif command == 'test':
        cmd_test(context)


In [ ]:
run_main('train')

Loaded 2775 axial slices for the training set.


Loaded 1656 axial slices for the validation set.


/home/GRAMES.POLYMTL.CA/p112175/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1 training loss: 7.0737.
Epoch 1 validation loss: 7.6085.
Epoch 1 took 111.83 seconds.
Epoch 2 training loss: 7.6030.
Epoch 2 validation loss: 7.6085.
Epoch 2 took 111.97 seconds.
Epoch 3 training loss: 7.6098.
Epoch 3 validation loss: 7.6085.
Epoch 3 took 112.32 seconds.
Epoch 4 training loss: 7.6225.


# Testing

In [ ]:
run_main('test')

# Random testing

In [ ]:
with open('config_small.json') as fhandle:
    context = json.load(fhandle)

command = context["command"]

if command == 'train':
    cmd_train(context)
    shutil.copyfile(sys.argv[1], "./"+context["log_directory"]+"/config_file.json")
elif command == 'test':
    cmd_test(context)

In [14]:
preds = torch.rand(18,6)
labels = [4, 5, 1, 3, 2, 5, 2, 5, 0, 3, 1, 0, 4, 0, 3, 0, 5, 3]
var_labels = []
for l in labels :
    a = [0 for i in range(6)]
    a[l] = 1 
    var_labels.append(a)

var_labels = torch.FloatTensor(var_labels)
print(var_labels)
print(preds)
CS_loss = nn.BCELoss()
loss = CS_loss(var_labels, var_labels)
loss

tensor([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.]])
tensor([[0.8693, 0.5932, 0.4733, 0.7017, 0.0742, 0.4677],
        [0.1578, 0.3839, 0.9845, 0.6197, 0.9022, 0.4011],
        [0.2514, 0.1741, 0.7124, 0.3909, 0.0939, 0.5291],
        [0.3461, 0.2243, 0.6523, 0.1974, 0.0667, 0.2130],
        [0.2069, 0.6367, 0.4330, 0.9309, 0.1388, 0.8270],
        [0.2280, 0.8140, 0.5117, 0.6479, 0.1460, 0.7722],
        [0.4406, 0.9427, 0.9723, 0.6072

tensor(0.)